In [6]:
import logging
import sys

# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# sys.stdout=stdout

os.environ['NUMEXPR_MAX_THREADS'] = '16'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [7]:
block_path = tdd.get_tdx_dir_blocknew() + '060.blk'

In [8]:
def get_multi_date_duration(df, dt):
    dd = df.reset_index()
    dd = dd[dd.date >= dt]
    # dd['couts'] = dd.groupby(['code'])['code'].transform('count')
    dd = dd.set_index(['code', 'date'])
    return dd


def get_multi_code_count(df, col='code'):
    dd = df.reset_index()
    dd['couts'] = dd.groupby([col])[col].transform('count')
    # dd = dd.sort_values(by='couts',ascending=0)
    print('count dd.couts')
    dd = dd.set_index(['code', 'date'])
    return dd


def multindex_iloc(df, index):
    label = df.index.levels[0][index]
    return df.iloc[df.index.get_loc(label)]


def get_groupby_mean_median_close(dfs):

    groupd = dfs.groupby(level=[0])

    df = groupd['close'].agg(['median', 'mean'])
    df['close'] = groupd.tail(1).reset_index().set_index(['code'])['close']
    # dfs['mean'] = groupd['close'].agg('mean')
    # dfs['median'] = groupd['close'].agg('median')

    # dfs = dfs.fillna(0)
    # idx = pd.IndexSlice
    # mask = ( (dfs['mean'] > dfs['median'])
    #         & (dfs['close'] > dfs['mean'])
    #         )
    # df=dfs.loc[idx[mask, :]]

    df = df[(df['mean'] > df['median']) & (df['close'] > df['mean'])]

    # dt_low = None
    # if dl == 1:
    #     dfs = groupd.tail(1)
    #     print("dfs tail1")
    # else:
    #     dl = 30
    #     dindex = tdd.get_tdx_Exp_day_to_df(
    #         '999999', dl=dl).sort_index(ascending=False)
    #     dt = tdd.get_duration_price_date('999999', df=dindex)
    #     dt = dindex[dindex.index >= dt].index.values
    #     dt_low = dt[-1]
    #     dtlen = len(dt) if len(dt) >0 else 1
    #     dfs = groupd.tail(dtlen)
    #     print("dfs tail:%s dt:%s"%(dtlen,dt))
    #     dfs = get_multi_date_duration(dfs,dt[-1])
    return df

In [9]:
def get_roll_mean_all(single=True, tdx=False, app=True, duration=100, ma_250_l=1.02, ma_250_h=1.11, resample='d'):
    time_s = time.time()
    if not app and cct.get_file_size(block_path) > 100 and cct.creation_date_duration(block_path) == 0:
        print("It's Today Update")
        return True
    code_list = sina_data.Sina().market('all').index.tolist()
    code_list.extend(['999999', '399001', '399006'])
    print("all code:", len(code_list))
    if duration < 300:
        h5_fname = 'tdx_all_df' + '_' + str(300)
        h5_table = 'all' + '_' + str(300)
    else:
        h5_fname = 'tdx_all_df' + '_' + str(900)
        h5_table = 'all' + '_' + str(900)
    # df = tdd.search_Tdx_multi_data_duration('tdx_all_df_300', 'all_300', df=None,code_l=code_list, start='20150501', end=None, freq=None, col=None, index='date')
    df = tdd.search_Tdx_multi_data_duration(
        h5_fname, h5_table, df=None, code_l=code_list, start=None, end=None, freq=None, col=None, index='date')
    # df = tdd.search_Tdx_multi_data_duration(h5_fname, h5_table, df=None,code_l=code_list, start=None, end=None, freq=None, col=None, index='date',tail=1)

    code_uniquelist = df.index.get_level_values('code').unique()

    code_select = code_uniquelist[random.randint(0, len(code_uniquelist) - 1)]
    print(round(time.time() - time_s, 2), df.index.get_level_values(
        'code').unique().shape, code_select, df.loc[code_select].shape)
    # df.groupby(level=[0]),df.index.get_level_values(0)
    # len(df.index.get_level_values('code').unique())
    # df = df[~df.index.duplicated(keep='first')]

    # multiIndex_func = {'close': 'mean', 'low': 'min', 'high': 'max', 'volume': 'sum', 'open': 'first'}
    # cct.using_Grouper(df, freq='W', col={'close': 'last'})
    # *** TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'
    # roll_dl = duration

    if resample != 'd':
        print("resample:%s" % (resample.upper()))
        # ohlc_dict = {'volume': 'sum','high': 'max','low': 'min','close': 'last'}
        # {'low': 'min', 'close':'mean'}
        # df = df.groupby(level=0).resample(resample, level=1).agg({'close':'last'})
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(level=0).resample(resample, level=1).agg({'close':'last'})
        df = df.loc[:, ['close']].groupby(level=0).resample(
            resample, level=1).agg({'close': 'last'}).dropna()

        df = df.dropna()
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(level=0).resample(resample, level=1, how={'low': 'min', 'close':'mean'})
        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(pd.Grouper(freq=resample)).transform(np.cumsum).resample(resample, how='ohlc')

        # df = df.loc[:, ['open','close', 'low', 'high']].groupby(pd.TimeGrouper(resample)).transform(np.cumsum).resample(resample, how='ohlc')
        # r1 = len(df.loc['000001'])
        # r2 = len(df.loc['999999'])
        # roll_dl = r1 if r1 < r2 else r2

        # df['upper'] = map(lambda x: round((1 + 11.0 / 100) * x, 1), df.ma10d)
        # df['lower'] = map(lambda x: round((1 - 9.0 / 100) * x, 1), df.ma10d)
        # df['ene'] = map(lambda x, y: round((x + y) / 2, 1), df.upper, df.lower)
        # dfs['amount%d'%cumdays] = groupd['amount'].apply(pd.rolling_mean, cumdays)
    # df.ix[df.index.levels[0]]
    # df.ix[df.index[len(df.index)-1][0]] #last row
    # dfs = tdd.search_Tdx_multi_data_duration(df=dfs,code_l=code_list, start='20170918', end='20170918', freq=None, col=None, index='date')

    # print dfs[:1],len(dfs)
    # groupd.agg({'low': 'min'})
    # '''idx mask filter'''
    # '''
    # h5a.write_hdf_db('all300', df, table='roll200', index=False,baseCount=500, append=False, MultiIndex=MultiIndex)
    print("use time:%s" % (round(time.time() - time_s, 2)))
    return df

In [10]:
resample = 'w'

df = get_roll_mean_all(single=False, tdx=True, app=True,
                       duration=250, ma_250_l=1.02, ma_250_h=1.2, resample=resample)

all code: 4957
0.94 (4542,) 002168 (616, 6)
resample:W


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

In [6]:
df.info()
df.describe()
df.loc['000001']

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 295496 entries, ('000001', Timestamp('2021-11-14 00:00:00')) to ('999999', Timestamp('2023-02-05 00:00:00'))
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   close   295496 non-null  float64
dtypes: float64(1)
memory usage: 3.4+ MB


,close
count,295496.000000
mean,26.741779
std,194.680269
min,0.930000
25%,6.370000
50%,12.050000
75%,23.690000
max,15111.560000


,close
date,
2021-11-14,18.04
2021-11-21,17.92
2021-11-28,17.35
2021-12-05,17.42
2021-12-12,18.15
...,...
2023-01-01,13.16
2023-01-08,14.62
2023-01-15,14.95


In [7]:
app = True
tdx = True
duration = 250
ma_250_l = 1.02
ma_250_h = 1.2
single = False

time_s = time.time()
dfs = df.copy()
groupd = dfs.groupby(level=[0])
# groupd.['close']
# rollma = ['5','10','60','100','200']
# rollma = ['5','10','250']
# df.index.get_level_values('code')[0]
if resample.upper() == 'D' or resample.lower() == 'd':
    if duration < 300:
        rollma = ['10', '200']
    else:
        rollma = ['10', '250']

elif resample.upper() == 'W' or resample.lower() == 'w':
    rollma = ['5', '10']
else:
    rollma = ['5', '10']

# rollma.extend([str(duration)])

# import ipdb;ipdb.set_trace()
# df.loc['300130'][:2]

# dfs['mean'] = groupd['close'].agg('mean')
# dfs['median'] = groupd['close'].agg('median')

for da in rollma:
    cumdays = int(da)
    dfs['ma%d' % cumdays] = groupd['close'].rolling(cumdays).mean().values
    if cumdays == 10:
        dfs['upper'] = dfs['ma%d' % cumdays].apply(
            lambda x: round((1 + 11.0 / 100) * x, 1))
        dfs['lower'] = dfs['ma%d' % cumdays].apply(
            lambda x: round((1 - 9.0 / 100) * x, 1))
        dfs['ene'] = list(map(lambda x, y: round(
            (x + y) / 2, 1), dfs['upper'], dfs['lower']))


dfs = dfs.dropna()

dt_low = None
df_idx = None
if single:
    dfs = groupd.tail(1)
    print("dfs tail1")
else:
    single = True
    dl = 30
    dindex = tdd.get_tdx_Exp_day_to_df(
        '999999', dl=dl, resample=resample).sort_index(ascending=False)
    dt = tdd.get_duration_price_date('999999', df=dindex)

    dt = dindex[dindex.index >= dt].index.values
    dt_low = dt[-1]
    dtlen = len(dt) if len(dt) > 0 else 1
    dfs = groupd.tail(dtlen)
    # import ipdb;ipdb.set_trace()
    df_idx = get_groupby_mean_median_close(dfs)

    print(("dfs tail:%s dt:%s" % (dtlen, dt)))
    dfs = get_multi_date_duration(dfs, dt[-1])

# groupd2 = dfs.groupby(level=[0])
# dfs['ma%d'%cumdays] = groupd['close'].apply(pd.rolling_mean, cumdays)

# dfs.reset_index().groupby(['code'])['date'].transform('count')
# single = True

dfs[:5]
dfs[('ma%s') % (rollma[-1])][:2]
print(df_idx)

dfs tail:5 dt:['2023-01-20' '2023-01-13' '2023-01-06' '2022-12-30' '2022-12-23']


close     ma5    ma10  upper  lower   ene
code   date                                                 
000001 2023-01-01  13.16  13.216  12.251   13.6   11.1  12.3
       2023-01-08  14.62  13.560  12.671   14.1   11.5  12.8
       2023-01-15  14.95  13.810  13.084   14.5   11.9  13.2
       2023-01-22  15.13  14.168  13.454   14.9   12.2  13.6
       2023-02-05  14.32  14.436  13.727   15.2   12.5  13.8

code    date      
000001  2023-01-01    12.251
        2023-01-08    12.671
Name: ma10, dtype: float64

        median     mean   close
code                           
000004    9.68    9.796   10.16
000012    6.88    6.948    7.20
000014   10.19   10.198   10.72
000016    4.66    4.674    4.80
000019    7.51    7.550    7.83
...        ...      ...     ...
688789  172.63  177.192  190.71
688798  105.45  108.692  118.21
688799   33.33   33.406   35.06
688981   41.68   42.008   43.33
689009   30.98   31.518   33.88

[2826 rows x 3 columns]


In [8]:
time_s = time.time()
dfs = dfs.fillna(0)
idx = pd.IndexSlice
# mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[0])])  & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])
# mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[1])])  & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])
# mask = (dfs[('ma%s')%(rollma[0])] > dfs[('ma%s')%(rollma[1])]) & (dfs[('ma%s')%(rollma[-1])] > 0) &  (dfs[('close')] > dfs[('ma%s')%(rollma[-1])])

# mask = ( (dfs[('ma%s')%(rollma[0])] > 0) & (dfs[('ma%s')%(rollma[-1])] > 0) & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])]) & (dfs[('close')] > dfs[('ma%s')%(rollma[0])]))
# mask = ( (dfs[('ma%s')%(rollma[0])] > 0) & (dfs[('ma%s')%(rollma[-1])] > 0)
#         & (dfs[('close')] > dfs[('ma%s')%(rollma[-1])]*ma_250_l)
#         & (dfs[('close')] < dfs[('ma%s')%(rollma[-1])]*ma_250_h)
#         & (dfs[('close')] > dfs[('ma%s')%(rollma[0])]))

# & (dfs['mean'] > dfs['median'])
# & (dfs['close'] > dfs['mean'])


if len(rollma) > 1:

    mask = ((dfs[('ma%s') % (rollma[0])] > 0) & (dfs[('ma%s') % (rollma[-1])] > 0) &
            (dfs[('ma%s') % (rollma[0])] > dfs[('ma%s') % (rollma[-1])])
            & (dfs[('close')] > dfs[('ma%s') % (rollma[0])])
            & (dfs[('close')] > dfs[('ma%s') % (rollma[-1])]*ma_250_h)
            & ((dfs[('close')] > dfs['ene']) | (dfs[('close')] > dfs['upper']))
            )
else:
    mask = ((dfs[('ma%s') % (rollma[0])] > 0)
            & (dfs[('close')] > dfs[('ma%s')%(rollma[0])])
            & ((dfs[('close')] > dfs['ene']) | (dfs[('close')] > dfs['upper']) )
            )


# mask = ((dfs[('close')] > dfs[('ma%s')%(rollma[-1])]))
df_mask = dfs.loc[idx[mask, :]]
print(df_mask.shape)
df_mask = get_multi_code_count(df_mask)
print((df_mask.couts[:5],df_mask.shape))

# import ipdb;ipdb.set_trace()
# df.sort_values(by='couts',ascending=0)
# groupd.first()[:2],groupd.last()[:2]
# groupd = df250.groupby(level=[0])
# '''
# groupd.transform(lambda x: x.iloc[-1])
# groupd.last()
# groupd.apply(lambda x: x.close > x.ma250)
# df.shape,df.sort_index(ascending=False)[:5]
# ?groupd.agg
# groupd = df.groupby(level=[0])
# groupd['close'].apply(pd.rolling_mean, 250, min_periods=1)
# ex:# Group df by df.platoon, then apply a rolling mean lambda function to df.casualties
# df.groupby('Platoon')['Casualties'].apply(lambda x:x.rolling(center=False,window=2).mean())

code_uniquelist = df_mask.index.get_level_values('code').unique()
code_select = code_uniquelist[random.randint(0, len(code_uniquelist) - 1)]

# if app:
#     print(round(time.time() - time_s, 2), 's', df.index.get_level_values(
#         'code').unique().shape, code_select, df.loc[code_select][-1:])

if single:
    # groupd = df.groupby(level=[0])
    if tdx:
        # block_path = tdd.get_tdx_dir_blocknew() + '060.blk'
        # if cct.get_work_time():
        #     codew = df[df.date == cct.get_today()].index.tolist()

        if dt_low is not None:

            groupd2 = df_mask.groupby(level=[0])
            df_mask = groupd2.tail(1)
            df_mask = df_mask.reset_index().set_index('code')
            # import ipdb;ipdb.set_trace()

            # df = df[(df.date >= dt_low) & (df.date <= cct.get_today())]
            dd = df_mask[(df_mask.date == dt_low)]
            # df = df[(df_mask.date >= cct.last_tddate(1))]
            df_mask = df_mask[(df_mask.date >= df_mask.date.max())]
            # import ipdb;ipdb.set_trace()
            print(("df:%s %s df_idx:%s" %
                  (len(df_mask), df_mask.index[:5], len(df_idx))))

            if df_idx is not None and len(df_mask) > 0 and len(df_idx) > 0:
                idx_set_ = [x for x in df_idx.index if x in df_mask.index]
                df_mask = df_mask.loc[idx_set_, :].dropna()
            print(("Main Down dd :%s MainUP df:%s couts std:%0.1f " %
                  (len(dd), len(df_mask), df_mask.couts.std())))
            # print df.date.mode()[0]
            df_mask = df_mask.sort_values(by='couts', ascending=1)
            df_mask = df_mask[df_mask.couts > df_mask.couts.std()]
            # df = df[(df.date >= df.date.mode()[0]) & (df.date <= cct.get_today())]
            codew = df_mask.index.tolist()

            if app:
                print("time: %s %s %s"%(round(time.time() - time_s, 2), 'groupd2', len(df_mask)))

        else:
            df_mask = df_mask.reset_index().set_index('code')
            df_mask = df_mask[(df_mask.date >= cct.last_tddate(days=10))
                    & (df_mask.date <= cct.get_today())]
            codew = df_mask.index.tolist()

        top_temp = tdd.get_sina_datadf_cnamedf(codew, df_mask)
        top_temp = top_temp[(~top_temp.index.str.contains(
            '688')) & (~top_temp.name.str.contains('ST'))]
        top_temp = top_temp.sort_values(by=['couts'],ascending=[0])
        codew = top_temp.index.tolist()

        # clean st and 688



    # df['date'] = df['date'].apply(lambda x:(x.replace('-','')))
    # df['date'] = df['date'].astype(int)
    # print df_mask.loc[code_select].T,df_mask.shape
#     MultiIndex = False
# else:
#     MultiIndex = True

(430, 6)
count dd.couts
(code    date      
000006  2023-01-01    1
000419  2023-01-01    1
000428  2023-01-01    1
000430  2023-01-01    1
000610  2023-01-01    2
Name: couts, dtype: int64, (430, 7))
df:185 Index(['000681', '000701', '000821', '000831', '000913'], dtype='object', name='code') df_idx:2826
Main Down dd :0 MainUP df:146 couts std:0.7 
time: 0.01 groupd2 146


In [9]:
top_temp

,date,close,ma5,ma10,upper,lower,ene,couts,name
code,,,,,,,,,
300585,2023-02-05,34.90,27.000,21.287,23.6,19.4,21.5,5,奥联电子
603255,2023-02-05,59.07,50.904,41.698,46.3,37.9,42.1,5,鼎际得
002235,2023-02-05,10.67,8.808,7.022,7.8,6.4,7.1,4,安妮股份
605168,2023-02-05,121.68,105.100,92.826,103.0,84.5,93.8,3,三人行
002279,2023-02-05,9.95,8.538,7.244,8.0,6.6,7.3,3,久其软件
...,...,...,...,...,...,...,...,...,...
603993,2023-02-05,5.89,5.014,4.872,5.4,4.4,4.9,1,洛阳钼业
605020,2023-02-05,49.99,43.174,41.421,46.0,37.7,41.9,1,永和股份
605133,2023-02-05,39.88,31.850,29.198,32.4,26.6,29.5,1,嵘泰股份


In [11]:
if app:
    print("Write blk:%s" % (block_path))
    hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
    if len(hdf5_wri) == 0:
        pass
    elif hdf5_wri == 'y' or hdf5_wri == 'Y':
        append_status = False
    else:
        append_status = True
else:
    append_status = False

if hdf5_wri == 'y' and len(codew) > 10:
    cct.write_to_blocknew(
        block_path, codew, append_status, doubleFile=False, keep_last=0)
    print("write:%s block_path:%s" % (len(codew), block_path))
else:
    print("write error:%s block_path:%s" % (len(codew), block_path))

Write blk:D:\MacTools\WinTools\new_tdx\T0002\blocknew\060.blk


rewrite code [Y] or append [N]: y


all write to D:\MacTools\WinTools\new_tdx\T0002\blocknew\060.blk:115
write:115 block_path:D:\MacTools\WinTools\new_tdx\T0002\blocknew\060.blk
